In [2]:
import os
import numpy as np
import torch
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel
import math
import torch.nn as nn
import os
import numpy as np
import re
# import matplotlib
import argparse
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import copy
import math
from tqdm.auto import tqdm,trange
from torch.nn import SyncBatchNorm
from torch.utils.data.dataloader import default_collate
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
# from torch.utils.tensorboard import SummaryWriter
import datasets
import json
import evaluate
from datasets import load_dataset, load_from_disk
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
dataset

Using the latest cached version of the module from D:\ProgramData\huggingface\modules\datasets_modules\datasets\cnn_dailymail\1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de (last modified on Sat Mar 11 16:11:44 2023) since it couldn't be found locally at cnn_dailymail., or remotely on the Hugging Face Hub.
Found cached dataset cnn_dailymail (D:/ProgramData/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
from nltk.tokenize import sent_tokenize
sent_tokenize(dataset["train"][0]["article"])

["LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
 'Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.',
 '"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.',
 '"I don\'t think I\'ll be particularly extravagant.',
 '"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."',
 'At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box off

In [15]:
def get_prompt_article(example):
    from nltk.tokenize import sent_tokenize
    article_sents = sent_tokenize(example['article'])
    highlights_sents = sent_tokenize(example['highlights'])
    if len(highlights_sents) == 1:
        highlights_sents = sent_tokenize(example['highlights'].replace("\n", ". "))
    if len(highlights_sents) == 1:
        highlights_sents = sent_tokenize(example['highlights'].replace("  ", ". "))

    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    for i, article_sent in enumerate(article_sents):
        rouges = []
        for j, highlights_sent in enumerate(highlights_sents):
            score = scorer.score(highlights_sent, article_sent)["rouge1"].fmeasure
            rouges.append((j + 1, score))
        rouges = sorted(rouges, key=lambda x: x[1], reverse=True)
        if rouges[0][1] > 0:
            article_sents[i] = f'[{rouges[0][0]}] {article_sents[i]} [/{rouges[0][0]}]'
    example['article'] = ' '.join(article_sents)
    return example

In [16]:
ds = dataset.map(get_prompt_article, num_proc=20)

In [5]:
from nltk.tokenize import sent_tokenize
sent_tokenize(dataset["train"][0]["highlights"])

['Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .',
 'Young actor says he has no plans to fritter his cash away .',
 "Radcliffe's earnings from first five Potter films have been held in trust fund ."]

In [17]:
ds["train"][0]["article"]

'[1] LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. [/1] [2] Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. [/2] [1] "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. [/1] "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." [3] At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the

In [19]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scorer.score('Young actor says he has no plans to fritter his cash away .', '"But I try very hard not to go that way because it would be too easy for them."')

{'rouge1': Score(precision=0.05555555555555555, recall=0.08333333333333333, fmeasure=0.06666666666666667),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.05555555555555555, recall=0.08333333333333333, fmeasure=0.06666666666666667)}